---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

use v1.11 to pass this course smoothly because the grader is using this NetworkX version to execute your code for assignment
! pip install networkx==1.11

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def degree_distribution(G):
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes( G)) for i in degree_values]
    return histogram

In [4]:
degree_distribution(P1_Graphs[1])

[0.002, 0.023, 0.166, 0.615, 0.171, 0.022, 0.001]

In [5]:
nx.average_clustering(P1_Graphs[1])

0.5642419635919628

In [6]:
nx.average_shortest_path_length(P1_Graphs[1])

5.089871871871872

In [7]:
for G in P1_Graphs:
    print(nx.average_clustering(G), nx.average_shortest_path_length(G), len(degree_distribution(G)))

0.03167539146454044 4.099161161161161 29
0.5642419635919628 5.089871871871872 7
0.4018222222222227 9.378702269692925 5
0.03780379975223251 3.1048046283934134 37
0.0033037037037037037 5.0785509568313305 9


the Preferential Attachment model generates graphs with the node degrees following  the power-law distribution (since rich gets richer), the graphs with this pattern in their degree distributions are most likely generated by this model.

On the other hand, the Small World model generates graphs with the node degrees not following the power-law distribution, instead the distribution shows fat tails. If this pattern is seen, we can identify the network as to be generated with this model.

In [8]:
def graph_identification():
    methods = []
    for G in P1_Graphs:
        clustering = nx.average_clustering(G)
        shortest_path = nx.average_shortest_path_length(G)
        degree_hist = degree_distribution(G)
        if len(degree_hist)>10:
            methods.append('PA')
        elif clustering < 0.1:
            methods.append('SW_H')
        else:
            methods.append('SW_L')
    return methods

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [9]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


In [10]:
G.nodes(data=True)[:10]

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (2, {'Department': 21, 'ManagementSalary': nan}),
 (3, {'Department': 21, 'ManagementSalary': 1.0}),
 (4, {'Department': 21, 'ManagementSalary': 1.0}),
 (5, {'Department': 25, 'ManagementSalary': nan}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (7, {'Department': 14, 'ManagementSalary': 0.0}),
 (8, {'Department': 14, 'ManagementSalary': nan}),
 (9, {'Department': 14, 'ManagementSalary': 0.0})]

In [11]:
G.node[2]['Department']

21

In [12]:
G.node[2]['ManagementSalary']

nan

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [13]:
def salary_predictions():

    #convert the graph to df
    email_df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
    email_df['degree'] = pd.Series(G.degree())
    email_df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    email_df['clustering'] = pd.Series(nx.clustering(G))
    email_df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    email_df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    email_df['pr'] = pd.Series(nx.pagerank(G))
    #ManagementSalary_df=pd.DataFrame(email_df["ManagementSalary"].value_counts())
    email_df_train = email_df[~pd.isnull(email_df['ManagementSalary'])]
    email_df_test = email_df[pd.isnull(email_df['ManagementSalary'])]

    X = email_df_train.drop('ManagementSalary', axis=1)
    y = email_df_train.ManagementSalary

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_test_ = email_df_test.drop('ManagementSalary', axis=1)
    X_scaled_test_ = scaler.transform(X_test_)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=0)

    # random forest Python
    clf_model = RandomForestClassifier(min_samples_split=100, max_depth=5, random_state=0,class_weight={0:5, 1:1}).fit(X_train, y_train)

    #get predictions
    y_pred = clf_model.predict(X_test)

    # predict probabilities
    y_probs = clf_model.predict_proba(X_test)

    # keep probabilities for the positive outcome only
    y_probs = y_probs[:, 1]

    # calculate AUC
    auc_ = roc_auc_score(y_test, y_probs)

    # get probability for testing df
    probability = clf_model.predict_proba(X_scaled_test_)[:,1]
    final_result = pd.Series(probability, X_test_.index)

    return final_result

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [14]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

• Link prediction problem: Given a network, predict which edges will be formed in the future.

• 5 basic measures:
    – Number of Common Neighbors
    – Jaccard Coefficient
    – Resource Allocation Index
    – Adamic-Adar Index
    – Preferential Attachment Score
    
• 2 measures that require community information:
    – Common Neighbor Soundarajan-Hopcroft Score
    – Resource Allocation Soundarajan-Hopcroft Score
    Department indicates the department in the company which the person belongs to, this can be considered as the community

In [15]:
def new_connections_predictions():
    
    common_neighbors = [(e[0], e[1], len(list(nx.common_neighbors(G, e[0], e[1])))) for e in nx.non_edges(G)]
    feature_df = pd.DataFrame(index=[(x[0], x[1]) for x in common_neighbors])

    feature_df['common_neighbors'] = [x[2] for x in common_neighbors]
    feature_df['preferential_attachment'] = [x[2] for x in list(nx.preferential_attachment(G))]
    feature_df['adamic_adar_index'] = [x[2] for x in list(nx.adamic_adar_index(G))]
    feature_df['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    feature_df['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G))]

    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']

    feature_df['cn_soundarajan_hopcroft'] = [x[2] for x in list(nx.cn_soundarajan_hopcroft(G))]
    feature_df['ra_soundarajan_hopcroft'] = [x[2] for x in list(nx.ra_index_soundarajan_hopcroft(G))]

    #feature_df.columns[feature_df.isna().any()].tolist()

    new_connections = pd.merge(future_connections, feature_df, left_index = True,right_index=True, how='outer')
    new_connections.rename(columns = {'Future Connection': 'Future_Connection'}, inplace = True)

    #new_connections.columns[new_connections.isna().any()].tolist()

    #Future_df=pd.DataFrame(new_connections["Future_Connection"].value_counts())
    connection_df_train = new_connections[~pd.isnull(new_connections['Future_Connection'])]
    connection_df_test = new_connections[pd.isnull(new_connections['Future_Connection'])]

    X = connection_df_train.drop('Future_Connection', axis=1)
    y = connection_df_train.Future_Connection

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_test_ = connection_df_test.drop('Future_Connection', axis=1)
    X_scaled_test_ = scaler.transform(X_test_)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=0)

    # random forest Python
    clf_model = RandomForestClassifier(min_samples_split=100, max_depth=5, random_state=0,class_weight={0:12, 1:1}).fit(X_train, y_train)

    #get predictions
    y_pred = clf_model.predict(X_test)

    # predict probabilities
    y_probs = clf_model.predict_proba(X_test)

    # keep probabilities for the positive outcome only
    y_probs = y_probs[:, 1]

    # calculate AUC
    auc_ = roc_auc_score(y_test, y_probs)

    # get probability for testing df
    probability = clf_model.predict_proba(X_scaled_test_)[:,1]
    predictions = pd.Series(probability, X_test_.index)

    return predictions# Your Answer Here